In [150]:
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
from collections import Counter
import pickle
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
import re

consumer_key = 'JyrZ4DEjv6NPw4da7x3VEX1lM'
consumer_secret = 'ACA0VFQLWntdWuC9GUyZ2VM7IopMjJ8LAskUnvWD6z4F9E6uKz'
access_token = '771134752929767424-VN16texnckjaNOj5Fe1278mdCyNU3ry'
access_token_secret = 'pXmyxBHg2lZ914WUaQu8itCSSg32iDYgdF8Psqcb6TgP6'

def get_twitter():
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.
    """
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)

In [383]:
def get_tweets():
    twitter = get_twitter()
    trump_tweets = []
    hillary_tweets = []
    n_tweets=500
    for r in twitter.request('statuses/filter', {'track': 'trump', 'language':'en'}):
        if 'text' in r.keys():
            trump_tweets.append(r)
            if len(trump_tweets) % 100 == 0:
                print('%d tweets' % len(trump_tweets))
            if len(trump_tweets) >= n_tweets:
                break
                
    for r2 in twitter.request('statuses/filter', {'track': 'hillary', 'language':'en'}):
        if 'text' in r2.keys():
            hillary_tweets.append(r2)
            if len(hillary_tweets) % 100 == 0:
                print('%d tweets' % len(hillary_tweets))
            if len(hillary_tweets) >= n_tweets:
                break
                
#    for r1 in twitter.request('statuses/filter', {'track': 'hillary', 'language':'en'}):
#        if 'text' in r1.keys():
#            tweets_hillary.append(r1)
#            if len(tweets) % 100 == 0:
#                print('%d tweets' % len(tweets))
#            if len(tweets_hillary) >= n_tweets:
#                break
    
    save_file('trump_tweets.txt',trump_tweets)
    save_file('hillary_tweets.txt',hillary_tweets)
get_tweets()

100 tweets
200 tweets
300 tweets
400 tweets
500 tweets
100 tweets
200 tweets
300 tweets
400 tweets
500 tweets


In [127]:
inputfile = open('tweets.txt','rb')
tweets = pickle.load(inputfile)
#text = [t['text'] for t in tweets]
#print(text)
for t in tweets:
    print(t['text'])
#print(len(tweets))
#print([t['user']['screen_name'] for t in tweets])
#loc = [t['user']['location'] for t in tweets]
#c1 = Counter(loc)
#print(c1.most_common(10))

RT @KaraCalavera: I want to let Trump know every day of his presidency that he lost the popular vote by 2.5 million votes. He loves r… 
Según Wilbur Ross secretario de Comercio Trump, venezuela puede ser pieza clave https://t.co/XDWPdFzLrC 95  !
RT @PrisonPlanet: Yet ANOTHER hate crime. Man beaten by mob because they suspected he voted for Trump. https://t.co/F63wErGmAN
Trump: Companies won't leave U.S. 'without consequences' #video https://t.co/BnB5lqu3Hx https://t.co/pojTgLF1y1
RT @Mediaite: Fox’s Gutfeld: If Obama Took ‘Victory Lap’ Like Trump, ‘This Network Would Explode’ https://t.co/XNxY4PxfZQ (VIDEO) https://t…
RT @GartrellLinda: This Is Fantastic - Watch Donald Trump in the Greatest Christmas Carol Ever https://t.co/dRQGnoPWES
MERRY CHRISTMAS!
Hil…
RT @Rockprincess818: There's no difference between Obama going on the View to discuss his personal and political views &amp; Trump Tweeting the…
RT @WeNeedTrump: Even with a rigged election, Trump won 3,084 out of 3,141 counties in t

KeyError: 'text'

In [232]:
def save_file(filename,data):
    out = open(filename,'ab+')
#    tweets = get_tweets()
    pickle.dump(data,out)
    out.close()
    
    

In [233]:
def load_file(filename):
    inputfile = open(filename,'rb')
    data = pickle.load(inputfile)
    return data
    

In [151]:
def afinn_down():
    url = urlopen('http://www2.compute.dtu.dk/~faan/data/AFINN.zip')
    zipfile = ZipFile(BytesIO(url.read()))
    afinn_file = zipfile.open('AFINN/AFINN-111.txt')
    afinn = dict()
    for line in afinn_file:
        parts = line.strip().split()
        if len(parts) == 2:
            afinn[parts[0].decode("utf-8")] = int(parts[1])
    save_file('afinn_data.txt',afinn)
    
afinn_down()


In [384]:
def sentiment_analysis():
    tokens = []
    trump_tweets = load_file('trump_tweets.txt')
    hillary_tweets = load_file('hillary_tweets.txt')
    afinn = load_file('afinn_data.txt')
    trump_tokens = tokenize(trump_tweets)
    hillary_tokens = tokenize(hillary_tweets)
    trump_positives = []
    trump_negatives = []
    hillary_positives = []
    hillary_negatives = []
    for token_list, tweet in zip(trump_tokens, trump_tweets):
        pos, neg = afinn_score(token_list, afinn)
        if pos > neg1:
            trump_positives.append((tweet['text'], pos, neg))
                
        elif neg > pos:
            trump_negatives.append((tweet['text'], pos, neg))
    
    save_file('trump_positive.txt',trump_positives)
    save_file('trump_negative.txt',trump_negatives)
    
    for token_list, tweet in zip(hillary_tokens, hillary_tweets):
        pos, neg = afinn_score(token_list, afinn)
        if pos > neg:
            hillary_positives.append((tweet['text'], pos, neg))
                
        elif neg > pos:
            hillary_negatives.append((tweet['text'], pos, neg))
    
    save_file('hillary_positive.txt',hillary_positives)
    save_file('hillary_negative.txt',hillary_negatives)
    
    
#    for tweet, pos, neg in sorted(negatives, key=lambda x: x[1], reverse=True):
#        print(pos, neg, tweet)
        
sentiment_analysis()
    

In [358]:
def afinn_score(terms, afinn):
    pos = 0
    neg = 0
    for t in terms:
        if t in afinn:
            if afinn[t] > 0:
                pos += afinn[t]
            else:
                neg += -1 * afinn[t] 
    return pos, neg

In [225]:
def tokenize(tweets):
    tokens = []
    for tweet in tweets:
        text = tweet['text'].lower()
        text = re.sub('@\S+', ' ', text)
        text = re.sub('http\S+', ' ', text)
        tokens.append(re.findall('[A-Za-z]+', text))
    return tokens

TwitterRequestError: Twitter request failed (429)

In [409]:
def cluster():
    trump_tweets = load_file('trump_tweets.txt')
    hillary_tweets = load_file('hillary_tweets.txt')
    
    trump_tweets1 = [t for t in trump_tweets if 'user' in t]
    hillary_tweets1 = [t for t in hillary_tweets if 'user' in t]
    
    for t in trump_tweets1:
        G.add_node(t['user']['screen_name'])
    for h in hillary_tweets1:
        G.add_node(h['user']['screen_name'])
        
    for x in range(0, len(trump_tweets1)):
        G.add_edge(trump_tweets1[x]['user']['screen_name'], trump_tweets1[x+1]['user']['screen_name'])
    
    for x in range(0, len(hillary_tweets1)):
        G.add_edge(hillary_tweets1[x]['user']['screen_name'], hillary_tweets1[x+1]['user']['screen_name'])
        
    nx.draw()
    plt.savefig('graph.png')


In [408]:
def girvan_newman(G, most_valuable_edge=None):
    if G.number_of_edges() == 0:
        yield tuple(nx.connected_components(G))
        return
    if most_valuable_edge is None:
        def most_valuable_edge(G):
            betweenness = nx.edge_betweenness_centrality(G)
            return max(betweenness, key=betweenness.get)
    g = G.copy().to_undirected()
    g.remove_edges_from(g.selfloop_edges())
    while g.number_of_edges() > 0:
        yield _without_most_central_edges(g, most_valuable_edge)
def _without_most_central_edges(G, most_valuable_edge):
    original_num_components = nx.number_connected_components(G)
    num_new_components = original_num_components
    while num_new_components <= original_num_components:
        edge = most_valuable_edge(G)
        G.remove_edge(*edge)
        new_components = tuple(nx.connected_components(G))
        num_new_components = len(new_components)
    return new_components

In [ ]:
def summarize():
    trump_tweets = load_file('trump_tweets.txt')
    hillary_tweets = load_file('hillary_tweets.txt')
    hillary_positives = load_file('hillary_positive.txt')
    hillary_negatives = load_file('hillary_negative.txt')
    trump_positives = load_file('trump_positive.txt')
    trump_negatives = load_file('trump_negative.txt')
    
    trump_tweets1 = [t for t in trump_tweets if 'user' in t]
    hillary_tweets1 = [t for t in hillary_tweets if 'user' in t]
    
    total = len(trump_tweets1) + len(hillary_tweets1)
    
    save_file('summary.txt','Total Users :')
    save_file('summary.txt',total)
    
    save_file('summary.txt','No of messages :')
    save_file('summary.txt',len(trump_tweets) + len(hillary_tweets))
    
    res = []
    for tweet, pos, neg in sorted(hillary_positives, key=lambda x: x[1], reverse=True):
        top pos = pos
        top neg = neg
        t = tweet
        res.append(pos,neg,tweet)
        save_file('summary.txt','Top positive sentiment tweet for Hillary :')
        save_file('summary.txt',res)
    
        break
        
    for tweet, pos, neg in sorted(hillary_negatives, key=lambda x: x[1], reverse=True):
        top pos = pos
        top neg = neg
        t = tweet
        res.append(pos,neg,tweet)
        save_file('summary.txt','Top negative sentiment tweet for Hillary :')
        save_file('summary.txt',res)
        break
        
    for tweet, pos, neg in sorted(trump_positives, key=lambda x: x[1], reverse=True):
        top pos = pos
        top neg = neg
        t = tweet
        res.append(pos,neg,tweet)
        save_file('summary.txt','Top positive sentiment tweet for Trump :')
        save_file('summary.txt',res)
        break
        
    for tweet, pos, neg in sorted(trump_negatives, key=lambda x: x[1], reverse=True):
        top pos = pos
        top neg = neg
        t = tweet
        res.append(pos,neg,tweet)
        save_file('summary.txt','Top negative sentiment tweet for Trump :')
        save_file('summary.txt',res)
        break
        
        
    